**Status: The qcodes dataset changes are still fresh. Proceed at your device's risk.**

# Installation
From [the docs](http://qcodes.github.io/Qcodes/start/index.html):

1. Install [Anaconda](https://www.anaconda.com/download/). Get the Python 3 version. No need to install the VSCode stuff.
1. Save [environment.yml](https://raw.githubusercontent.com/QCoDeS/Qcodes/master/environment.yml) to the machine.
1. Run the following in an Anaconda Prompt:
```
conda env create -f environment.yml
activate qcodes
pip install qcodes
```
1. Now you can run `jupyter notebook` from within your qcodes environment.

In [ ]:
%matplotlib notebook
import time
import numpy as np
import matplotlib.pyplot as plt
import qcodes as qc
from qcodes.dataset.measurements import Measurement
from qcodes.dataset.plotting import plot_by_id
from qcodes.dataset.data_export import get_data_by_id

# Instrument setup

At this point, each measurement setup needs to define which instruments they use. See [the docs](http://qcodes.github.io/Qcodes/user/intro.html#instrument). For demonstration purposes, here I just use dummy instruments `dac` and `dmm`.

In [ ]:
from qcodes.tests.instrument_mocks import DummyInstrument
dac = DummyInstrument(name="dac", gates=['ch1', 'ch2'])
dmm = DummyInstrument(name="dmm", gates=['idc', 'ig'])

import random
dmm.idc.get = lambda: random.gauss(1, 0.1)
dmm.ig.get = lambda: random.gauss(0, 0.01)

station = qc.Station(dac, dmm)

# Experiment setup

The first time you run qcodes on a new computer, you need to create the SQLite database. See [the docs](https://qcodes.github.io/Qcodes/dataset/index.html) for details on the design of the dataset and database. You will then want to make a new experiment to hold your data. At the very least, make a new experiment when you change out your samples.

In [ ]:
qc.initialise_database()
qc.new_experiment(name='demo', sample_name='my best sample')

# Measurement

## 1D sweep

TODO: Improve live plotting.

In [ ]:
# Setpoints specify which parameters are independent and which are dependent.
# For a 1D sweep, we will have 1 independent parameter and any number of dependent parameters.
meas = Measurement()
meas.register_parameter(dac.ch1)
meas.register_parameter(dmm.ig, setpoints=(dac.ch1,))
meas.register_parameter(dmm.idc, setpoints=(dac.ch1,))

# These are totally optional, but can be useful.
meas.add_before_run(lambda: print('This happens first.'), ())
meas.add_before_run(lambda dmm: print(f'This happens next: {dmm.idc.get()}'), (dmm,))
meas.add_after_run(lambda: print('This happens last.'), ())

# This, along with the plot update logic, are terrible. Maybe the qcodes people know better?
fig, axs = plt.subplots(1, 2, figsize=(8, 4))
axs[0].set_xlabel(f'{dac.ch1.label} ({dac.ch1.unit})')
axs[0].set_ylabel(f'{dmm.ig.label} ({dmm.ig.unit})')
axs[1].set_xlabel(f'{dac.ch1.label} ({dac.ch1.unit})')
axs[1].set_ylabel(f'{dmm.idc.label} ({dmm.idc.unit})')
lines = [ax.plot([], [])[0] for ax in axs]
fig.show()

with meas.run() as datasaver:
    for set_v in np.linspace(0, 25, 10):
        dac.ch1.set(set_v)
        ig = dmm.ig.get()
        idc = dmm.idc.get()
        
        datasaver.add_result((dac.ch1, set_v), (dmm.ig, ig), (dmm.idc, idc))
        
        # Plot update logic. It's bad.
        lines[0].set_xdata(np.append(lines[0].get_xdata(), set_v))
        lines[0].set_ydata(np.append(lines[0].get_ydata(), ig))
        lines[1].set_xdata(np.append(lines[1].get_xdata(), set_v))
        lines[1].set_ydata(np.append(lines[1].get_ydata(), idc))
        for ax in axs:
            ax.relim()
            ax.autoscale_view()
        fig.tight_layout()
        fig.canvas.draw()

## Continuously measure

TODO: Is this try/catch method actually safe? Check `datasaver.add_result`, and maybe check the getters?

In [ ]:
# Let's register a custom independent parameter called 'time'.
# This does not need to be a qcodes parameter!
meas = Measurement()
meas.register_custom_parameter('time', label='Time', unit='s')
meas.register_parameter(dmm.ig, setpoints=('time',))

fig, ax = plt.subplots(1, 1, figsize=(4, 4))
ax.set_xlabel('Time (s)')
ax.set_ylabel(f'{dmm.ig.label} ({dmm.ig.unit})')
lines = ax.plot([], [])[0]
fig.show()

with meas.run() as datasaver:     
    try:
        while True:
            t = time.monotonic()
            ig = dmm.ig.get()
            datasaver.add_result(('time', t), (dmm.ig, ig))
            
            lines.set_xdata(np.append(lines.get_xdata(), t))
            lines.set_ydata(np.append(lines.get_ydata(), ig))
            ax.relim()
            ax.autoscale_view()
            fig.tight_layout()
            fig.canvas.draw()
            
            time.sleep(1)
    except KeyboardInterrupt:
        pass

## 2D sweep

TODO: Estimate total time. Live plotting.

In [ ]:
meas = Measurement()
meas.register_parameter(dac.ch1)
meas.register_parameter(dac.ch2)
meas.register_parameter(dmm.ig, setpoints=(dac.ch1, dac.ch2))

with meas.run() as datasaver:
    try:
        for set_ch1 in np.linspace(0, 25, 10):
            for set_ch2 in np.linspace(0, 10, 10):
                dac.ch1.set(set_ch1)
                dac.ch2.set(set_ch2)
                ig = dmm.ig.get()

                datasaver.add_result((dac.ch1, set_ch1), (dac.ch2, set_ch2), (dmm.ig, ig))
    except KeyboardInterrupt:
        pass
    dataid = datasaver.run_id
plot_by_id(dataid)

## Where is the data?

See the `get_data_by_id` docstring.

TODO: Make this easier to save to a csv or something.

In [ ]:
get_data_by_id(dataid)